In [ ]:
import pandas as pd
import tabula 

In [ ]:
# UBICACIÓN DEL ARCHIVO PDF - EXTRACTO BANCARIO PROVINCIA
ruta = "D:\DOCUMENTOS\kari\PDF Kari\ExtractoBancario2.pdf"

# DETERMINAR EL CBU DE LA CUENTA BANCARIA
CBU = tabula.read_pdf(
    ruta, 
    pages=1,
    relative_area=True, 
    relative_columns=True,
    area=[10, 0, 16, 100],
    columns=[65], 
    pandas_options={'header': None}) [0][1][1]

In [ ]:
# LECTURA DE TABLAS DESDE EL PDF USANDO TABULA
dfs = tabula.read_pdf(
    ruta, 
    pages='all',
    relative_area=True, 
    relative_columns=True,
    area=[20, 0, 95, 100],
    columns=[10, 55, 73, 80], 
    silent=True)

In [ ]:
# RENOMBRAR COLUMNAS DE LAS TABLAS
nuevos_nombres = ['Fecha', 'Concepto', 'Importe', 'Valor', 'Saldo']
dfs_renombrados = [df.rename(columns=dict(zip(df.columns, nuevos_nombres))) for df in dfs]

# UNIR TODAS LAS TABLAS EN UN SOLO DATAFRAME
df_unido = pd.concat(dfs_renombrados, ignore_index=True)

In [ ]:
# FILTRAR FILAS CON FECHA NO VACÍA Y FORMATO 'dd-mm-aa'
df_filtro = df_unido[df_unido['Fecha'].notna() & df_unido['Fecha'].str.match(r'^\d{2}-\d{2}-\d{2}$')]

# CREAR COLUMNAS DE DÉBITOS Y CRÉDITOS
df_filtro['Debitos'] = df_filtro['Importe'].astype(str).apply(lambda x: x if x.startswith('-') else None)
df_filtro['Creditos'] = df_filtro['Importe'].astype(str).apply(lambda x: x if not x.startswith('-') else None)

In [ ]:
# OBTENER FECHAS INICIAL Y FINAL DEL EXTRACTO BANCARIO
desde = df_filtro['Fecha'].iloc[0]
hasta = df_filtro['Fecha'].iloc[-1]
fecha_desde = "-".join(reversed(desde.split("-")))
fecha_hasta = "-".join(reversed(hasta.split("-")))

# GENERAR NOMBRE DEL ARCHIVO CSV DE SALIDA
nombre_archivo = CBU + "_Periodo_" + fecha_desde + " al " + fecha_hasta + ".csv"

In [ ]:
# ORDENAR LAS COLUMNAS EN EL DATAFRAME FINAL
df_orden = df_filtro['Fecha', 'Concepto', 'Importe', 'Valor', 'Debitos', 'Creditos', 'Saldo']

# GUARDAR EL DATAFRAME EN UN ARCHIVO CSV
df_orden.to_csv(nombre_archivo, encoding='latin-1', sep=';', index=False)